In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
# from IBA.tensorflow_v1 import IBALayer
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)
from sklearn.metrics import classification_report
from keras.utils.np_utils import *
from keras.callbacks import LearningRateScheduler
from sklearn.preprocessing import LabelBinarizer, label_binarize
from keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import SGD, Adam
from models.models import VGG16Net, VGG19Net, ResNet50Net
from models.VGG import VGG16Net_IBA
import matplotlib.pyplot as plt
import utils_paths
import numpy as np
import random
import pickle
import cv2
import os


Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
def scheduler(epoch):

    if epoch % 100 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)

def load_cla_data(train_imagePaths, target):

    data = []
    labels = []

    for imagePath in train_imagePaths:

        image = cv2.imread(imagePath)
        image = cv2.resize(image, target)
        data.append(image)
        print(imagePath)
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    data = np.array(data, dtype="float")
    #data = data.swapaxes(1, 3)
    data = data / 255.0
    labels = np.array(labels)

    return data, labels

In [3]:
INIT_LR = 1e-3
EPOCHS = 100
BS = 8
width = 224
height = 224
depth = 3
target = (width, height)
Name = "VGG16Net_IBA_DES"
GPU = True
if GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 建立卷积神经网络
# model = ResNet50Net.build(width=width, height=height, depth=depth, classes=2)
model = VGG16Net_IBA.build(width=width, height=height, depth=depth, classes=2)
# model = VGG16Net.build(width=width, height=height, depth=depth, classes=2)
# model = VGG19Net.build(width=width, height=height, depth=depth, classes=2)
# model = VGG16Net_IBA.build(width=width, height=height, depth=depth, classes=len(lb.classes_))
#model = load_model('weights_VGG16_ICIS_100-0.8378.h5')
model.summary()



Instructions for updating:
Colocations handled automatically by placer.


2022-05-09 06:34:04.191995: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-09 06:34:04.229321: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-05-09 06:34:04.233623: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5afe740 executing computations on platform Host. Devices:
2022-05-09 06:34:04.233712: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-05-09 06:34:04.715648: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5baa7a0 executing computations on platform CUDA. Devices:
2022-05-09 06:34:04.715739: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-05-09 06:34:04.716899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

IndexError: list index out of range

In [ ]:
print("------data loading-----")
train_path = 'dataset/DES_aug/train/images/'# train_path = './dataset/OCT/train/'
test_path = train_path.replace('train', 'test')
val_path = train_path.replace('train', 'val')

train_imagePaths = sorted(list(utils_paths.list_images(train_path)))
val_imagePaths = sorted(list(utils_paths.list_images(val_path)))
test_imagePaths = sorted(list(utils_paths.list_images(test_path)))
random.seed(307)
random.shuffle(train_imagePaths)
random.shuffle(val_imagePaths)
random.shuffle(test_imagePaths)


train_x, train_y = load_cla_data(train_imagePaths, target)
val_x, val_y = load_cla_data(val_imagePaths, target)
test_x, test_y = load_cla_data(test_imagePaths, target)

lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
val_y = lb.fit_transform(val_y)
test_y = lb.fit_transform(test_y)
# train_y = label_binarize(train_y, np.arange(2))
train_y = to_categorical(train_y, 2)
val_y = to_categorical(val_y, 2)
test_y = to_categorical(test_y, 2)

# 数据增强处理
# aug = ImageDataGenerator(rotation_range=30,
#                          # brightness_range=(0.5, 0.9),
#                          # rescale=0.8,
#                          shear_range=0.2,
#                          zoom_range=0.1,
#                          horizontal_flip=True,
#                          # vertical_flip=True,
#                          fill_mode="nearest"
#                          )

In [ ]:
print("------begin training -----")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=1e-5, nesterov=True)
#opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# opt = Adam(lr=INIT_LR)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_lr=1e-8, mode='auto', verbose=1)

checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{val_acc:.4f}.h5',
                                     monitor='val_acc', mode='auto', save_best_only='True', period=20)

checkpoint_period2 = ModelCheckpoint(Name + '-{epoch:03d}-{val_acc:.4f}.h5',
                                     monitor='val_acc', mode='auto', period=20)

# earlyStopping = EarlyStopping(monitor='val_acc', patience=20, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, min_lr=1e-8, mode='auto', verbose=1)
#
#
hist = model.fit(train_x, train_y,
                 batch_size=BS,
                 epochs=EPOCHS,
                 validation_data=(val_x, val_y),
                 class_weight="auto",
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger])

# hist = model.fit_generator(aug.flow(train_x, train_y, batch_size=BS),
#                            validation_data=aug.flow(val_x, val_y),
#                            steps_per_epoch=len(train_x) // BS,
#                            class_weight={0: 0.8, 1: 1.2},
#                            validation_steps=5,
#                            workers=6,
#                            epochs=EPOCHS, callbacks=[checkpoint_period1, checkpoint_period2, csv_logger, reduce_lr])

In [ ]:
print("------Saving model------")
# model.save('vgg16_150_ICIS_1-700X700.h5')
model_filename = Name + ".h5"
model.save_weights(model_filename)
print('model saved to:', model_filename)


print("------ testing ------")
predictions = model.predict(test_x, batch_size=8)
print(classification_report(test_y.argmax(axis=1),
      predictions.argmax(axis=1),
      target_names=lb.classes_,
      digits=6))


In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, hist.history["loss"], label="train_loss")
plt.plot(N, hist.history["val_loss"], label="val_loss")
plt.plot(N, hist.history["acc"], label="train_acc")
plt.plot(N, hist.history["val_acc"], label="val_acc")
# plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('./output/' + Name + '_plot.png')